Here's a rough outline of what we'd like our graph to look like:
![](https://i.imgur.com/5TedoE1.jpg)

**Note:**

- Each edge now has either a ```[None]``` or ```[]``` icon next to it. This represents the TensorFlow shape of the tensor flowing through that edge, with ```None``` representing a vector of any length, and ```[]``` representing a scalar.

- The output of node ```d``` now flows into an update section, which contains Operations necessary to update Variables as well as pass data through to the TensorBoard summaries.

- We have a separate name scope to contain our two Variables
    - one to store the accumulated sum of our outputs
    - the other to keep track of how many times we've run the graph. Since these two Variables operate outside of the flow of our main transformation, it makes sense to put them in a separate space.
    
- There is a name scope dedicated to our TensorBoard summaries which will hold our ```tf.scalar_summary``` Operations. We place them after the "update" section to ensure that the summaries are added after we update our Variables, otherwise things could run out of order.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Explicitly create the graph that we would like to use
# instead of using the default graph
graph = tf.Graph()

In [3]:
with graph.as_default():
    with tf.name_scope("variables"):
        # set trainable=False since we aren' training anything!
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, 
                                  trainable=False, 
                                  name="global_step")
        
        # Variable that keeps track of sum of all output values over time
        total_output = tf.Variable(0.0, dtype=tf.float32, 
                                   trainable=False, 
                                   name="total_output")
        
    with tf.name_scope("transformation"):
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder - takes in a vector
            a = tf.placeholder(tf.float32, shape=[None], 
                               name="input_placeholder_a")
            
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
            
        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
            
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)
        
        # Increments the above 'global_step' variable,
        # Should be run whenever the graph is run
        increment_step = global_step.assign_add(1)
    
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32),
                    name="average")
        
        # Creates summaries for output node
        tf.summary.scalar("output_summary", output)
        tf.summary.scalar("total_summary", update_total)
        tf.summary.scalar("average_summary", avg)
        
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()
        
        # Merge all summaries into one operation
        merged_summaries = tf.summary.merge_all()

In [4]:
# Running the graph
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

- With a Session started, let's initialize our Variables before doing anything else:

In [5]:
sess.run(init)

In [6]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, 
                                 merged_summaries],
                                feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [7]:
# Let's actually sue it!
run_graph([2, 8])
run_graph([3, 1, 3, 3])
run_graph([8])
run_graph([1, 2, 3])
run_graph([11, 4])
run_graph([7, 3, 1])
run_graph([6, 3])

Write teh summaries to disk with the ```tf.summary.flush()```

In [8]:
writer.flush()

In [9]:
writer.close()
sess.close()

![](https://i.imgur.com/sfzjg4P.jpg)